#Paytm Labs Data Challenge

####Problem Statement:
Using the global weather data, answer the following:
1. Which country had the hottest average mean temperature over the year?
2. Which country had the most consecutive days of tornadoes/funnel cloud
formations?
3. Which country had the second highest average mean wind speed over the year?

In [0]:
# Importing necessary packages
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *

In [0]:
# Constants
file_format = "csv"
file_header = "true"
file_path = "/FileStore/tables"
file_delimiter = ","
stations_path = file_path + "/stationlist.csv"
countries_path = file_path + "/countrylist.csv"
data_path = file_path + "/*.gz"

root
-- STN_NO: string (nullable = true)
-- COUNTRY_ABBR: string (nullable = true)

root
-- COUNTRY_ABBR: string (nullable = true)
-- COUNTRY_FULL: string (nullable = true)

In [0]:
#Loading Stations
station_df = spark.read.format(file_format) \
  .options(header=file_header, delimiter = ',') \
  .options(inferSchema="true")\
  .load(stations_path) 

#Loading countries
country_df = spark.read.format(file_format) \
  .options(header=file_header, delimiter = ',') \
  .options(inferSchema="true")\
  .load(countries_path) 

station_df.printSchema()
country_df.printSchema()

In [0]:
#Schema for data DF
data_schema = StructType() \
      .add("STN---",IntegerType(),True) \
      .add("WBAN",IntegerType(),True) \
      .add("YEARMODA",IntegerType(),True) \
      .add("TEMP",DoubleType(),True) \
      .add("DEWP",DoubleType(),True) \
      .add("SLP",DoubleType(),True) \
      .add("STP",DoubleType(),True) \
      .add("VISIB",DoubleType(),True) \
      .add("WDSP",DoubleType(),True) \
      .add("MXSPD",DoubleType(),True) \
      .add("GUST",DoubleType(),True) \
      .add("MAX",DoubleType(),True) \
      .add("MIN",DoubleType(),True) \
      .add("PRCP",DoubleType(),True) \
      .add("SNDP",DoubleType(),True) \
      .add("FRSHTT",StringType(),True) 

#Loading data DF
data_df = spark.read.format(file_format) \
  .options(header=file_header, delimiter = ',') \
  .schema(data_schema)\
  .load(data_path) 

data_df.printSchema()

root
-- STN---: integer (nullable = true)
-- WBAN: integer (nullable = true)
-- YEARMODA: integer (nullable = true)
-- TEMP: double (nullable = true)
-- DEWP: double (nullable = true)
-- SLP: double (nullable = true)
-- STP: double (nullable = true)
-- VISIB: double (nullable = true)
-- WDSP: double (nullable = true)
-- MXSPD: double (nullable = true)
-- GUST: double (nullable = true)
-- MAX: double (nullable = true)
-- MIN: double (nullable = true)
-- PRCP: double (nullable = true)
-- SNDP: double (nullable = true)
-- FRSHTT: string (nullable = true)

In [0]:
data_df.show()

+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+
STN---| WBAN|YEARMODA|TEMP|DEWP| SLP| STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN|PRCP| SNDP|FRSHTT|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+
 10260|99999|20190101|26.1|21.2|1001.9| 987.5| 20.6| 9.0| 15.9| 29.7|29.8|null|null| 18.5|001000|
 10260|99999|20190102|24.9|22.1|1020.1|1005.5| 5.4| 5.6| 13.6| 22.1|null|20.7|null| 22.8|001000|
 10260|99999|20190103|31.7|29.1|1008.9| 994.7| 13.6|11.6| 21.4| 49.5|null|null|null|999.9|011000|
 10260|99999|20190104|32.9|30.3|1011.4| 997.1| 15.8| 4.9| 7.8| 10.9|36.1|31.8|null|999.9|001000|
 10260|99999|20190105|35.5|33.0|1015.7|1001.4| 12.0|10.4| 13.6| 21.0|null|32.7|null| 23.6|010000|
 10260|99999|20190106|38.5|34.1|1008.2| 994.2| 12.8|10.0| 17.5| 28.9|41.4|null|null| 23.2|010000|
 10260|99999|20190107|32.1|29.8| 996.8| 982.7| 6.9|11.3| 15.5| 28.6|null|30.4|null|999.9|001000|
 10260|99999|20190108|31.6|28.0| 997.4| 983.3| 22.9| 5.9| 11.7| 19.0|34.3|null|null| 0.4|011000|
 10260|99999|20190109|29.9|27.7|1011.6| 997.3| 29.8| 7.6| 15.2| 26.6|32.4|26.1|null| 23.6|001000|
 10260|99999|20190110|33.1|30.6| 979.1| 965.3| 5.3|17.8| 24.9| 41.8|41.4|null|null|999.9|011000|
 10260|99999|20190111|31.2|29.0| 975.0| 961.1| 5.6|11.6| 17.5| 38.9|null|null|null| 0.4|011100|
 10260|99999|20190112|28.3|26.1| 988.2| 974.1| 8.2| 8.1| 13.6| 38.5|null|null|null|999.9|001000|
 10260|99999|20190113|22.7|20.9| 977.1| 963.0| 26.6| 4.1| 7.8| 15.2|27.7|null|null| 0.4|001000|
 10260|99999|20190114|20.0|18.3| 984.3| 970.0| 43.1| 3.6| 9.7| 10.7|null|15.4|null| 38.6|000000|
 10260|99999|20190115|25.9|23.2| 991.3| 977.1| 16.0| 7.4| 13.8| 20.8|27.3|19.2|null|999.9|001000|
 10260|99999|20190116|24.8|21.8| 992.5| 978.2| 33.4| 2.7| 5.8|999.9|26.1|23.5|null| 35.4|001000|
 10260|99999|20190117|21.4|19.0| 989.8| 975.5| 10.4| 6.1| 8.9| 13.4|null|null|null| 35.0|001000|
 10260|99999|20190118|21.0|19.1| 994.4| 980.0| 13.8| 5.6| 7.8| 12.8|22.3|19.2|null| 35.0|001000|
 10260|99999|20190119|20.2|18.5|1000.8| 986.3| 33.9| 3.4| 7.8| 10.3|null|null|null| 35.8|001000|
 10260|99999|20190120|21.7|18.5|1009.0| 994.4| 32.1| 9.5| 14.6| 20.4|null|18.7|null|999.9|001000|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+
only showing top 20 rows

In [0]:
#Not Caching - Using once
data_df.count()

Out[27]: 4258415

In [0]:
#Joining stations DF with country DF while broadcasting country DF since it is smaller in size when compared to stations
station_country_df = station_df.join(country_df.hint("broadcast"), "COUNTRY_ABBR")
station_country_df.show()

+------------+------+--------------+
COUNTRY_ABBR|STN_NO| COUNTRY_FULL|
+------------+------+--------------+
 NO|012240| NORWAY|
 SW|020690| SWEDEN|
 SW|020870| SWEDEN|
 SW|021190| SWEDEN|
 UK|032690|UNITED KINGDOM|
 UK|033450|UNITED KINGDOM|
 UK|039290|UNITED KINGDOM|
 EI|039790| IRELAND|
 IC|040480| ICELAND|
 IC|041300| ICELAND|
 FO|060100| FAROE ISLANDS|
 DA|061443| DENMARK|
 NL|063401| NETHERLANDS|
 FR|071910| FRANCE|
 GM|092640| GERMANY|
 PL|123766| POLAND|
 PL|125990| POLAND|
 HU|129700| HUNGARY|
 HR|132240| CROATIA|
 BU|156500| BULGARIA|
+------------+------+--------------+
only showing top 20 rows

In [0]:
#Joining data DF with station_country DF while broadcasting station_country DF since it is smaller in size when compared to data DF

#First converitng STN_NO to match data_df STN_NO
station_country_df = station_country_df.withColumn('STN_NO', station_country_df['STN_NO'].cast('int'))

#Making the join
global_weather_data_df = data_df.join(station_country_df.hint("broadcast"), station_country_df["STN_NO"] == data_df['STN---'])
global_weather_data_df.show()

+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+------------+------+------------+
STN---| WBAN|YEARMODA|TEMP|DEWP| SLP| STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN|PRCP| SNDP|FRSHTT|COUNTRY_ABBR|STN_NO|COUNTRY_FULL|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+------------+------+------------+
 10260|99999|20190101|26.1|21.2|1001.9| 987.5| 20.6| 9.0| 15.9| 29.7|29.8|null|null| 18.5|001000| NO| 10260| NORWAY|
 10260|99999|20190102|24.9|22.1|1020.1|1005.5| 5.4| 5.6| 13.6| 22.1|null|20.7|null| 22.8|001000| NO| 10260| NORWAY|
 10260|99999|20190103|31.7|29.1|1008.9| 994.7| 13.6|11.6| 21.4| 49.5|null|null|null|999.9|011000| NO| 10260| NORWAY|
 10260|99999|20190104|32.9|30.3|1011.4| 997.1| 15.8| 4.9| 7.8| 10.9|36.1|31.8|null|999.9|001000| NO| 10260| NORWAY|
 10260|99999|20190105|35.5|33.0|1015.7|1001.4| 12.0|10.4| 13.6| 21.0|null|32.7|null| 23.6|010000| NO| 10260| NORWAY|
 10260|99999|20190106|38.5|34.1|1008.2| 994.2| 12.8|10.0| 17.5| 28.9|41.4|null|null| 23.2|010000| NO| 10260| NORWAY|
 10260|99999|20190107|32.1|29.8| 996.8| 982.7| 6.9|11.3| 15.5| 28.6|null|30.4|null|999.9|001000| NO| 10260| NORWAY|
 10260|99999|20190108|31.6|28.0| 997.4| 983.3| 22.9| 5.9| 11.7| 19.0|34.3|null|null| 0.4|011000| NO| 10260| NORWAY|
 10260|99999|20190109|29.9|27.7|1011.6| 997.3| 29.8| 7.6| 15.2| 26.6|32.4|26.1|null| 23.6|001000| NO| 10260| NORWAY|
 10260|99999|20190110|33.1|30.6| 979.1| 965.3| 5.3|17.8| 24.9| 41.8|41.4|null|null|999.9|011000| NO| 10260| NORWAY|
 10260|99999|20190111|31.2|29.0| 975.0| 961.1| 5.6|11.6| 17.5| 38.9|null|null|null| 0.4|011100| NO| 10260| NORWAY|
 10260|99999|20190112|28.3|26.1| 988.2| 974.1| 8.2| 8.1| 13.6| 38.5|null|null|null|999.9|001000| NO| 10260| NORWAY|
 10260|99999|20190113|22.7|20.9| 977.1| 963.0| 26.6| 4.1| 7.8| 15.2|27.7|null|null| 0.4|001000| NO| 10260| NORWAY|
 10260|99999|20190114|20.0|18.3| 984.3| 970.0| 43.1| 3.6| 9.7| 10.7|null|15.4|null| 38.6|000000| NO| 10260| NORWAY|
 10260|99999|20190115|25.9|23.2| 991.3| 977.1| 16.0| 7.4| 13.8| 20.8|27.3|19.2|null|999.9|001000| NO| 10260| NORWAY|
 10260|99999|20190116|24.8|21.8| 992.5| 978.2| 33.4| 2.7| 5.8|999.9|26.1|23.5|null| 35.4|001000| NO| 10260| NORWAY|
 10260|99999|20190117|21.4|19.0| 989.8| 975.5| 10.4| 6.1| 8.9| 13.4|null|null|null| 35.0|001000| NO| 10260| NORWAY|
 10260|99999|20190118|21.0|19.1| 994.4| 980.0| 13.8| 5.6| 7.8| 12.8|22.3|19.2|null| 35.0|001000| NO| 10260| NORWAY|
 10260|99999|20190119|20.2|18.5|1000.8| 986.3| 33.9| 3.4| 7.8| 10.3|null|null|null| 35.8|001000| NO| 10260| NORWAY|
 10260|99999|20190120|21.7|18.5|1009.0| 994.4| 32.1| 9.5| 14.6| 20.4|null|18.7|null|999.9|001000| NO| 10260| NORWAY|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+------------+------+------------+
only showing top 20 rows

In [0]:
#Caching for reuse
global_weather_data_df.cache().count()

Out[33]: 4143167

In [0]:
#Splitting Second FRSHTT column for individual values 
split_frshtt = split(col('FRSHTT'), "")
global_weather_data_df = global_weather_data_df\
          .withColumn('Fog', split_frshtt.getItem(0)) \
          .withColumn('Rain or Drizzle', split_frshtt.getItem(1))\
          .withColumn('Snow or Ice Pellets', split_frshtt.getItem(2))\
          .withColumn('Hail', split_frshtt.getItem(3))\
          .withColumn('Thunder', split_frshtt.getItem(4))\
          .withColumn('Tornado or Funnel', split_frshtt.getItem(5))
      
global_weather_data_df.printSchema()  

root
-- STN---: integer (nullable = true)
-- WBAN: integer (nullable = true)
-- YEARMODA: integer (nullable = true)
-- TEMP: double (nullable = true)
-- DEWP: double (nullable = true)
-- SLP: double (nullable = true)
-- STP: double (nullable = true)
-- VISIB: double (nullable = true)
-- WDSP: double (nullable = true)
-- MXSPD: double (nullable = true)
-- GUST: double (nullable = true)
-- MAX: double (nullable = true)
-- MIN: double (nullable = true)
-- PRCP: double (nullable = true)
-- SNDP: double (nullable = true)
-- FRSHTT: string (nullable = true)
-- COUNTRY_ABBR: string (nullable = true)
-- STN_NO: integer (nullable = true)
-- COUNTRY_FULL: string (nullable = true)
-- Fog: string (nullable = true)
-- Rain or Drizzle: string (nullable = true)
-- Snow or Ice Pellets: string (nullable = true)
-- Hail: string (nullable = true)
-- Thunder: string (nullable = true)
-- Tornado or Funnel: string (nullable = true)

In [0]:
global_weather_data_df.show()

+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+------------+------+------------+---+---------------+-------------------+----+-------+-----------------+
STN---| WBAN|YEARMODA|TEMP|DEWP| SLP| STP|VISIB|WDSP|MXSPD| GUST| MAX| MIN|PRCP| SNDP|FRSHTT|COUNTRY_ABBR|STN_NO|COUNTRY_FULL|Fog|Rain or Drizzle|Snow or Ice Pellets|Hail|Thunder|Tornado or Funnel|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+------------+------+------------+---+---------------+-------------------+----+-------+-----------------+
 10260|99999|20190101|26.1|21.2|1001.9| 987.5| 20.6| 9.0| 15.9| 29.7|29.8|null|null| 18.5|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190102|24.9|22.1|1020.1|1005.5| 5.4| 5.6| 13.6| 22.1|null|20.7|null| 22.8|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190103|31.7|29.1|1008.9| 994.7| 13.6|11.6| 21.4| 49.5|null|null|null|999.9|011000| NO| 10260| NORWAY| 0| 1| 1| 0| 0| 0|
 10260|99999|20190104|32.9|30.3|1011.4| 997.1| 15.8| 4.9| 7.8| 10.9|36.1|31.8|null|999.9|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190105|35.5|33.0|1015.7|1001.4| 12.0|10.4| 13.6| 21.0|null|32.7|null| 23.6|010000| NO| 10260| NORWAY| 0| 1| 0| 0| 0| 0|
 10260|99999|20190106|38.5|34.1|1008.2| 994.2| 12.8|10.0| 17.5| 28.9|41.4|null|null| 23.2|010000| NO| 10260| NORWAY| 0| 1| 0| 0| 0| 0|
 10260|99999|20190107|32.1|29.8| 996.8| 982.7| 6.9|11.3| 15.5| 28.6|null|30.4|null|999.9|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190108|31.6|28.0| 997.4| 983.3| 22.9| 5.9| 11.7| 19.0|34.3|null|null| 0.4|011000| NO| 10260| NORWAY| 0| 1| 1| 0| 0| 0|
 10260|99999|20190109|29.9|27.7|1011.6| 997.3| 29.8| 7.6| 15.2| 26.6|32.4|26.1|null| 23.6|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190110|33.1|30.6| 979.1| 965.3| 5.3|17.8| 24.9| 41.8|41.4|null|null|999.9|011000| NO| 10260| NORWAY| 0| 1| 1| 0| 0| 0|
 10260|99999|20190111|31.2|29.0| 975.0| 961.1| 5.6|11.6| 17.5| 38.9|null|null|null| 0.4|011100| NO| 10260| NORWAY| 0| 1| 1| 1| 0| 0|
 10260|99999|20190112|28.3|26.1| 988.2| 974.1| 8.2| 8.1| 13.6| 38.5|null|null|null|999.9|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190113|22.7|20.9| 977.1| 963.0| 26.6| 4.1| 7.8| 15.2|27.7|null|null| 0.4|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190114|20.0|18.3| 984.3| 970.0| 43.1| 3.6| 9.7| 10.7|null|15.4|null| 38.6|000000| NO| 10260| NORWAY| 0| 0| 0| 0| 0| 0|
 10260|99999|20190115|25.9|23.2| 991.3| 977.1| 16.0| 7.4| 13.8| 20.8|27.3|19.2|null|999.9|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190116|24.8|21.8| 992.5| 978.2| 33.4| 2.7| 5.8|999.9|26.1|23.5|null| 35.4|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190117|21.4|19.0| 989.8| 975.5| 10.4| 6.1| 8.9| 13.4|null|null|null| 35.0|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190118|21.0|19.1| 994.4| 980.0| 13.8| 5.6| 7.8| 12.8|22.3|19.2|null| 35.0|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190119|20.2|18.5|1000.8| 986.3| 33.9| 3.4| 7.8| 10.3|null|null|null| 35.8|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
 10260|99999|20190120|21.7|18.5|1009.0| 994.4| 32.1| 9.5| 14.6| 20.4|null|18.7|null|999.9|001000| NO| 10260| NORWAY| 0| 0| 1| 0| 0| 0|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+----+----+----+-----+------+------------+------+------------+---+---------------+-------------------+----+-------+-----------------+
only showing top 20 rows

#### Question 1: Which country had the hottest average mean temperature over the year?

In [0]:
#Check for any NULL values:
global_weather_data_df.filter(col('TEMP') == '9999.9').count()

Out[105]: 0

In [0]:
#Creating the Temperature DF
temperature_df = global_weather_data_df.select('STN---', 'YEARMODA', 'COUNTRY_FULL', 'TEMP' )
temperature_df.show()

+------+--------+------------+----+
STN---|YEARMODA|COUNTRY_FULL|TEMP|
+------+--------+------------+----+
 10260|20190101| NORWAY|26.1|
 10260|20190102| NORWAY|24.9|
 10260|20190103| NORWAY|31.7|
 10260|20190104| NORWAY|32.9|
 10260|20190105| NORWAY|35.5|
 10260|20190106| NORWAY|38.5|
 10260|20190107| NORWAY|32.1|
 10260|20190108| NORWAY|31.6|
 10260|20190109| NORWAY|29.9|
 10260|20190110| NORWAY|33.1|
 10260|20190111| NORWAY|31.2|
 10260|20190112| NORWAY|28.3|
 10260|20190113| NORWAY|22.7|
 10260|20190114| NORWAY|20.0|
 10260|20190115| NORWAY|25.9|
 10260|20190116| NORWAY|24.8|
 10260|20190117| NORWAY|21.4|
 10260|20190118| NORWAY|21.0|
 10260|20190119| NORWAY|20.2|
 10260|20190120| NORWAY|21.7|
+------+--------+------------+----+
only showing top 20 rows

In [0]:
temperature_df.groupBy('COUNTRY_FULL')\
              .agg(avg('TEMP').alias('Avg'))\
              .createOrReplaceTempView('countryTemperature')


In [0]:
%sql
select ct.COUNTRY_FULL, ct.Avg 
from countryTemperature ct
where ct.Avg IN (select MAX(ctt.Avg) from countryTemperature ctt)

COUNTRY_FULL,Avg
DJIBOUTI,90.06114457831325


The country which had the hottest average mean temperature over the year is __DJIBOUTI__

#### Question 3: Which country had the second highest average mean wind speed over the year?

In [0]:
#Check for any NULL values
global_weather_data_df.filter(col('TEMP') == '999.9').count()

Out[106]: 0

In [0]:
windspeed_df = global_weather_data_df.select('STN---', 'YEARMODA', 'COUNTRY_FULL', 'WDSP' )
windspeed_df.show()

+------+--------+------------+----+
STN---|YEARMODA|COUNTRY_FULL|WDSP|
+------+--------+------------+----+
 10260|20190101| NORWAY| 9.0|
 10260|20190102| NORWAY| 5.6|
 10260|20190103| NORWAY|11.6|
 10260|20190104| NORWAY| 4.9|
 10260|20190105| NORWAY|10.4|
 10260|20190106| NORWAY|10.0|
 10260|20190107| NORWAY|11.3|
 10260|20190108| NORWAY| 5.9|
 10260|20190109| NORWAY| 7.6|
 10260|20190110| NORWAY|17.8|
 10260|20190111| NORWAY|11.6|
 10260|20190112| NORWAY| 8.1|
 10260|20190113| NORWAY| 4.1|
 10260|20190114| NORWAY| 3.6|
 10260|20190115| NORWAY| 7.4|
 10260|20190116| NORWAY| 2.7|
 10260|20190117| NORWAY| 6.1|
 10260|20190118| NORWAY| 5.6|
 10260|20190119| NORWAY| 3.4|
 10260|20190120| NORWAY| 9.5|
+------+--------+------------+----+
only showing top 20 rows

In [0]:
windspeed_df.groupBy('COUNTRY_FULL')\
              .agg(avg('WDSP').alias('Avg'))\
              .createOrReplaceTempView('countryWindspeed')

In [0]:
%sql
select * from countryWindspeed

COUNTRY_FULL,Avg
SOUTH AFRICA,5.568036944632776
JAPAN,6.756455810670751
TANZANIA,5.190688184010396
BELARUS,153.37015143017527
LITHUANIA,25.487954059829082
UNITED KINGDOM,31.700467401221324
CZECH REPUBLIC,8.80699532741923
ESTONIA,49.47997024150183
CANADA,24.665389873002272
POLAND,8.217994430861172


In [0]:
%sql
select c.COUNTRY_FULL
from countryWindspeed c
where c.Avg IN (select MAX( cw.Avg )
                from countryWindspeed cw
                where cw.Avg NOT IN ( SELECT MAX( cww.Avg )
                                      FROM countryWindspeed cww
                                      )
                 )

COUNTRY_FULL
ARMENIA


The country which had the second highest average mean wind speed over the year is __ARMENIA__

#### Question 2: Which country had the most consecutive days of tornadoes/funnel cloud formations?

In [0]:
#Creating country with Tornado DF
countryTornado_df = global_weather_data_df.select('STN---', 'YEARMODA', 'COUNTRY_FULL', 'Tornado or Funnel' )
countryTornado_df.show()

+------+--------+------------+-----------------+
STN---|YEARMODA|COUNTRY_FULL|Tornado or Funnel|
+------+--------+------------+-----------------+
 10260|20190101| NORWAY| 0|
 10260|20190102| NORWAY| 0|
 10260|20190103| NORWAY| 0|
 10260|20190104| NORWAY| 0|
 10260|20190105| NORWAY| 0|
 10260|20190106| NORWAY| 0|
 10260|20190107| NORWAY| 0|
 10260|20190108| NORWAY| 0|
 10260|20190109| NORWAY| 0|
 10260|20190110| NORWAY| 0|
 10260|20190111| NORWAY| 0|
 10260|20190112| NORWAY| 0|
 10260|20190113| NORWAY| 0|
 10260|20190114| NORWAY| 0|
 10260|20190115| NORWAY| 0|
 10260|20190116| NORWAY| 0|
 10260|20190117| NORWAY| 0|
 10260|20190118| NORWAY| 0|
 10260|20190119| NORWAY| 0|
 10260|20190120| NORWAY| 0|
+------+--------+------------+-----------------+
only showing top 20 rows

In [0]:
from datetime import datetime

#Convert YEARMODA to yyyy-mm-dd format
func =  udf (lambda x: datetime.strptime(x, '%Y%m%d'), DateType())
countryTornado_df = countryTornado_df.withColumn('new_date', date_format(func(col('YEARMODA').astype('string')   ), 'yyyy-mm-dd'))

#Split the date into respective parts
split_date = split(col('new_date'), "-")
countryTornado_df = countryTornado_df.withColumn('year', split_date.getItem(0)) \
       .withColumn('month', split_date.getItem(1)) \
       .withColumn('day', split_date.getItem(2))
countryTornado_df.show()

+------+--------+------------+-----------------+----------+----+-----+---+
STN---|YEARMODA|COUNTRY_FULL|Tornado or Funnel| new_date|year|month|day|
+------+--------+------------+-----------------+----------+----+-----+---+
 10260|20190101| NORWAY| 0|2019-00-01|2019| 00| 01|
 10260|20190102| NORWAY| 0|2019-00-02|2019| 00| 02|
 10260|20190103| NORWAY| 0|2019-00-03|2019| 00| 03|
 10260|20190104| NORWAY| 0|2019-00-04|2019| 00| 04|
 10260|20190105| NORWAY| 0|2019-00-05|2019| 00| 05|
 10260|20190106| NORWAY| 0|2019-00-06|2019| 00| 06|
 10260|20190107| NORWAY| 0|2019-00-07|2019| 00| 07|
 10260|20190108| NORWAY| 0|2019-00-08|2019| 00| 08|
 10260|20190109| NORWAY| 0|2019-00-09|2019| 00| 09|
 10260|20190110| NORWAY| 0|2019-00-10|2019| 00| 10|
 10260|20190111| NORWAY| 0|2019-00-11|2019| 00| 11|
 10260|20190112| NORWAY| 0|2019-00-12|2019| 00| 12|
 10260|20190113| NORWAY| 0|2019-00-13|2019| 00| 13|
 10260|20190114| NORWAY| 0|2019-00-14|2019| 00| 14|
 10260|20190115| NORWAY| 0|2019-00-15|2019| 00| 15|
 10260|20190116| NORWAY| 0|2019-00-16|2019| 00| 16|
 10260|20190117| NORWAY| 0|2019-00-17|2019| 00| 17|
 10260|20190118| NORWAY| 0|2019-00-18|2019| 00| 18|
 10260|20190119| NORWAY| 0|2019-00-19|2019| 00| 19|
 10260|20190120| NORWAY| 0|2019-00-20|2019| 00| 20|
+------+--------+------------+-----------------+----------+----+-----+---+
only showing top 20 rows

In [0]:
w1 = Window.partitionBy('COUNTRY_FULL').orderBy('YEARMODA')
w2 = Window.partitionBy('COUNTRY_FULL', 'Tornado or Funnel').orderBy('YEARMODA')

res = countryTornado_df.withColumn('GROUP', row_number().over(w1)- row_number().over(w2))

#Window definition for streak to calculate the highest streaks
w3 = Window.partitionBy('COUNTRY_FULL', 'Tornado or Funnel', 'GROUP').orderBy('YEARMODA')
streak_res = res.withColumn('streak_0', when(col('Tornado or Funnel') == 1,0).otherwise(row_number().over(w3))) \
                .withColumn('streak_1', when(col('Tornado or Funnel') == 0,0).otherwise(row_number().over(w3)))


+------+--------+------------+-----------------+----------+----+-----+---+-----+--------+--------+
STN---|YEARMODA|COUNTRY_FULL|Tornado or Funnel| new_date|year|month|day|GROUP|streak_0|streak_1|
+------+--------+------------+-----------------+----------+----+-----+---+-----+--------+--------+
136240|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 1| 0|
136100|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 2| 0|
136010|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 3| 0|
136110|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 4| 0|
136000|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 5| 0|
136230|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 6| 0|
136250|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 7| 0|
136220|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 8| 0|
136290|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 9| 0|
136190|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 10| 0|
136150|20190101| ALBANIA| 0|2019-00-01|2019| 00| 01| 0| 11| 0|
136240|20190102| ALBANIA| 0|2019-00-02|2019| 00| 02| 0| 12| 0|
136100|20190102| ALBANIA| 0|2019-00-02|2019| 00| 02| 0| 13| 0|
136010|20190102| ALBANIA| 0|2019-00-02|2019| 00| 02| 0| 14| 0|
136110|20190102| ALBANIA| 0|2019-00-02|2019| 00| 02| 0| 15| 0|
136000|20190102| ALBANIA| 0|2019-00-02|2019| 00| 02| 0| 16| 0|
136230|20190102| ALBANIA| 0|2019-00-02|2019| 00| 02| 0| 17| 0|
136250|20190102| ALBANIA| 0|2019-00-02|2019| 00| 02| 0| 18| 0|
136220|20190102| ALBANIA| 0|2019-00-02|2019| 00| 02| 0| 19| 0|
136290|20190102| ALBANIA| 0|2019-00-02|2019| 00| 02| 0| 20| 0|
+------+--------+------------+-----------------+----------+----+-----+---+-----+--------+--------+
only showing top 20 rows